<a href="https://colab.research.google.com/github/ele9996/keyword-spotting/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
import os
from zipfile import ZipFile

In [28]:
import pathlib
pathlib.Path().absolute()

PosixPath('/content')

In [29]:
#parser = argparse.ArgumentParser()
#parser.add_argument('--model', type=str, required=True, help='model name')
#parser.add_argument('--mfcc', action='store_true', help='use MFCCs')
#parser.add_argument('--silence', action='store_true', help='add silence')
#args = parser.parse_args()

mfcc=False
silence=False

In [30]:
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

#Data extraction from file

In [31]:
#if args.silence is True:
if silence is True:
    data_dir = os.path.join('.', 'data', 'mini_speech_commands_silence')
else:
    zip_path = tf.keras.utils.get_file(
        origin="http://storage.googleapis.com/download.tensorflow.org/data/mini_speech_commands.zip",
        fname='mini_speech_commands.zip',
        extract=True,
        cache_dir='.', cache_subdir='data')

    data_dir = os.path.join('.', 'data', 'mini_speech_commands')

filenames = tf.io.gfile.glob(str(data_dir) + '/*/*')
filenames = tf.random.shuffle(filenames)
num_samples = len(filenames)

#if args.silence is True:
if silence is True:
    total = 9000
else:
    total = 8000

#Training, Validation and Test dataset definition

In [32]:
train_files = filenames[:int(total*0.8)]
val_files = filenames[int(total*0.8): int(total*0.9)]
test_files = filenames[int(total*0.9):]

In [33]:
LABELS = np.array(tf.io.gfile.listdir(str(data_dir)))
LABELS = LABELS[LABELS != 'README.md']

#Data preparation and preprocessing steps

In [34]:
class SignalGenerator:
    def __init__(self, labels, sampling_rate, frame_length, frame_step,
            num_mel_bins=None, lower_frequency=None, upper_frequency=None,
            num_coefficients=None, mfcc=False):
        self.labels = labels
        self.sampling_rate = sampling_rate
        self.frame_length = frame_length
        self.frame_step = frame_step
        self.num_mel_bins = num_mel_bins
        self.lower_frequency = lower_frequency
        self.upper_frequency = upper_frequency
        self.num_coefficients = num_coefficients
        num_spectrogram_bins = (frame_length) // 2 + 1

        if mfcc is True:
            self.linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(
                    self.num_mel_bins, num_spectrogram_bins, self.sampling_rate,
                    self.lower_frequency, self.upper_frequency)
            self.preprocess = self.preprocess_with_mfcc
        else:
            self.preprocess = self.preprocess_with_stft

    def read(self, file_path):
        parts = tf.strings.split(file_path, os.path.sep)
        label = parts[-2]
        label_id = tf.argmax(label == self.labels)
        audio_binary = tf.io.read_file(file_path)
        audio, _ = tf.audio.decode_wav(audio_binary)
        audio = tf.squeeze(audio, axis=1)

        return audio, label_id

    def pad(self, audio):
        zero_padding = tf.zeros([self.sampling_rate] - tf.shape(audio), dtype=tf.float32)
        audio = tf.concat([audio, zero_padding], 0)
        audio.set_shape([self.sampling_rate])

        return audio

    def get_spectrogram(self, audio):
        stft = tf.signal.stft(audio, frame_length=self.frame_length,
                frame_step=self.frame_step, fft_length=self.frame_length)
        spectrogram = tf.abs(stft)

        return spectrogram

    def get_mfccs(self, spectrogram):
        mel_spectrogram = tf.tensordot(spectrogram,
                self.linear_to_mel_weight_matrix, 1)
        log_mel_spectrogram = tf.math.log(mel_spectrogram + 1.e-6)
        mfccs = tf.signal.mfccs_from_log_mel_spectrograms(log_mel_spectrogram)
        mfccs = mfccs[..., :self.num_coefficients]

        return mfccs

    def preprocess_with_stft(self, file_path):
        audio, label = self.read(file_path)
        audio = self.pad(audio)
        spectrogram = self.get_spectrogram(audio)
        spectrogram = tf.expand_dims(spectrogram, -1)
        spectrogram = tf.image.resize(spectrogram, [32, 32])

        return spectrogram, label

    def preprocess_with_mfcc(self, file_path):
        audio, label = self.read(file_path)
        audio = self.pad(audio)
        spectrogram = self.get_spectrogram(audio)
        mfccs = self.get_mfccs(spectrogram)
        mfccs = tf.expand_dims(mfccs, -1)

        return mfccs, label

    def make_dataset(self, files, train):
        ds = tf.data.Dataset.from_tensor_slices(files)
        ds = ds.map(self.preprocess, num_parallel_calls=4)
        ds = ds.batch(32)
        ds = ds.cache()
        if train is True:
            ds = ds.shuffle(100, reshuffle_each_iteration=True)

        return ds

In [35]:
STFT_OPTIONS = {'frame_length': 256, 'frame_step': 128, 'mfcc': False}
MFCC_OPTIONS = {'frame_length': 640, 'frame_step': 320, 'mfcc': True,
        'lower_frequency': 20, 'upper_frequency': 4000, 'num_mel_bins': 40,
        'num_coefficients': 10}

In [36]:
#if args.mfcc is True:
if mfcc is True:
    options = MFCC_OPTIONS
    strides = [2, 1]
else:
    options = STFT_OPTIONS
    strides = [2, 2]


In [37]:
generator = SignalGenerator(LABELS, 16000, **options)
train_ds = generator.make_dataset(train_files, True)
val_ds = generator.make_dataset(val_files, False)
test_ds = generator.make_dataset(test_files, False)

#if args.silence is True:
if silence is True:
    units = 9
else:
    units = 8

#Model application

###MPL

In [38]:
cp_callback = keras.callbacks.ModelCheckpoint(
    filepath='./callback_val_chkp/stft_mpl',
    # './callback_test_chkp/chkp_best',
    monitor='val_loss',
    verbose=0, 
    save_best_only=True,
    # save_best_only=True,
    save_weights_only=False,
    mode='auto',
    save_freq='epoch'
)



In [39]:
#MLP Model definition
mlp_model = keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(units=256, activation='relu'),
    keras.layers.Dense(units=256, activation='relu'),
    keras.layers.Dense(units=256, activation='relu'),
    keras.layers.Dense(units=8) 
])

In [40]:
#Training, evaluation and testing with mlp model


mlp_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) ,optimizer='adam', metrics=['accuracy'])
mlp_model.fit(train_ds, epochs=20, validation_data=val_ds,callbacks=[cp_callback])



Epoch 1/20
200/200 [==============================] - 10s 52ms/step - loss: 1.5958 - accuracy: 0.4484 - val_loss: 1.2762 - val_accuracy: 0.5275
Epoch 2/20
200/200 [==============================] - 1s 7ms/step - loss: 1.0502 - accuracy: 0.6336 - val_loss: 1.0135 - val_accuracy: 0.6750
Epoch 3/20
200/200 [==============================] - 1s 7ms/step - loss: 0.8173 - accuracy: 0.7048 - val_loss: 0.9494 - val_accuracy: 0.7063
Epoch 4/20
200/200 [==============================] - 2s 9ms/step - loss: 0.6481 - accuracy: 0.7714 - val_loss: 0.8825 - val_accuracy: 0.7063
Epoch 5/20
200/200 [==============================] - 1s 4ms/step - loss: 0.5165 - accuracy: 0.8186 - val_loss: 0.9148 - val_accuracy: 0.7513
Epoch 6/20
200/200 [==============================] - 1s 7ms/step - loss: 0.4507 - accuracy: 0.8381 - val_loss: 0.8632 - val_accuracy: 0.7387
Epoch 7/20
200/200 [==============================] - 1s 7ms/step - loss: 0.3900 - accuracy: 0.8608 - val_loss: 0.8631 - val_accuracy: 0.7350
Epoc

In [41]:
#model evaluation on validation set
mlp_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer='adam')
val_loss= mlp_model.evaluate(val_ds)

25/25 [==============================] - 0s 2ms/step - loss: 1.6752


In [42]:
test_loss= mlp_model.evaluate(test_ds)

25/25 [==============================] - 2s 72ms/step - loss: 1.6573


###CNN

In [43]:
cp_callback = keras.callbacks.ModelCheckpoint(
    filepath='./callback_val_chkp/stft_cnn',
    # './callback_test_chkp/chkp_best',
    monitor='val_loss',
    verbose=0, 
    save_best_only=True,
    # save_best_only=True,
    save_weights_only=False,
    mode='auto',
    save_freq='epoch'
)


In [44]:
#CNN_2D Model definition

cnn_model = keras.Sequential([
    keras.layers.Conv2D(filters=128,kernel_size=[3,3],strides=strides, use_bias=False),
    keras.layers.BatchNormalization(momentum=0.1),
    keras.layers.ReLU(),
    keras.layers.Conv2D(filters=128,kernel_size=[3,3],strides=[1, 1], use_bias=False),
    keras.layers.BatchNormalization(momentum=0.1),
    keras.layers.ReLU(),
    keras.layers.Conv2D(filters=128,kernel_size=[3,3],strides=[1,1], use_bias=False),
    keras.layers.BatchNormalization(momentum=0.1),
    keras.layers.ReLU(),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(units=8)
])

In [45]:
#Training, evaluation and testing with mlp model
cnn_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) ,optimizer='adam', metrics=['accuracy'])
cnn_model.fit(train_ds, epochs=20, validation_data=val_ds,callbacks=[cp_callback])

Epoch 1/20
200/200 [==============================] - 44s 219ms/step - loss: 1.6011 - accuracy: 0.4469 - val_loss: 1.2167 - val_accuracy: 0.6288
Epoch 2/20
200/200 [==============================] - 44s 218ms/step - loss: 1.0479 - accuracy: 0.6775 - val_loss: 0.8772 - val_accuracy: 0.7262
Epoch 3/20
200/200 [==============================] - 44s 219ms/step - loss: 0.8270 - accuracy: 0.7470 - val_loss: 0.7617 - val_accuracy: 0.7650
Epoch 4/20
200/200 [==============================] - 44s 220ms/step - loss: 0.6800 - accuracy: 0.7984 - val_loss: 0.7511 - val_accuracy: 0.7675
Epoch 5/20
200/200 [==============================] - 44s 219ms/step - loss: 0.5776 - accuracy: 0.8341 - val_loss: 0.6014 - val_accuracy: 0.8125
Epoch 6/20
200/200 [==============================] - 43s 213ms/step - loss: 0.4993 - accuracy: 0.8536 - val_loss: 0.6104 - val_accuracy: 0.8037
Epoch 7/20
200/200 [==============================] - 44s 219ms/step - loss: 0.4392 - accuracy: 0.8736 - val_loss: 0.5323 - val_ac

In [46]:
#model evaluation on validation set
cnn_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer='adam')
val_loss= cnn_model.evaluate(val_ds)

25/25 [==============================] - 1s 49ms/step - loss: 0.9144


In [47]:
#model test on the test set
test_loss= cnn_model.evaluate(test_ds)

25/25 [==============================] - 1s 50ms/step - loss: 1.0850


###DS-CNN

In [48]:
cp_callback = keras.callbacks.ModelCheckpoint(
    filepath='./callback_val_chkp/stft_DS-CNN',
    # './callback_test_chkp/chkp_best',
    monitor='val_loss',
    verbose=0, 
    save_best_only=True,
    # save_best_only=True,
    save_weights_only=False,
    mode='auto',
    save_freq='epoch'
)


In [49]:
ds_cnn_model = keras.Sequential([
    keras.layers.Conv2D(filters=256,kernel_size=[3,3],strides=strides, use_bias=False),
    keras.layers.BatchNormalization(momentum=0.1),
    keras.layers.ReLU(),
    keras.layers.DepthwiseConv2D(kernel_size=[3,3],strides=[1, 1], use_bias=False),
    keras.layers.Conv2D(filters=256,kernel_size=[1,1],strides=[1,1], use_bias=False),
    keras.layers.BatchNormalization(momentum=0.1),
    keras.layers.ReLU(),
    keras.layers.DepthwiseConv2D(kernel_size=[3,3],strides=[1, 1], use_bias=False),
    keras.layers.Conv2D(filters=256,kernel_size=[1,1],strides=[1,1], use_bias=False),
    keras.layers.BatchNormalization(momentum=0.1),
    keras.layers.ReLU(),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(units=8)
])

In [50]:
#Training, evaluation and testing with mlp model
ds_cnn_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) ,optimizer='adam', metrics=['accuracy'])
ds_cnn_model.fit(train_ds, epochs=20, validation_data=val_ds,callbacks=[cp_callback])

Epoch 1/20
200/200 [==============================] - 35s 177ms/step - loss: 1.5476 - accuracy: 0.4667 - val_loss: 1.1940 - val_accuracy: 0.6438
Epoch 2/20
200/200 [==============================] - 35s 176ms/step - loss: 0.9959 - accuracy: 0.6883 - val_loss: 0.9016 - val_accuracy: 0.7225
Epoch 3/20
200/200 [==============================] - 35s 176ms/step - loss: 0.7663 - accuracy: 0.7666 - val_loss: 0.8139 - val_accuracy: 0.7287
Epoch 4/20
200/200 [==============================] - 35s 175ms/step - loss: 0.6274 - accuracy: 0.8119 - val_loss: 0.5489 - val_accuracy: 0.8300
Epoch 5/20
200/200 [==============================] - 35s 177ms/step - loss: 0.5284 - accuracy: 0.8417 - val_loss: 0.5262 - val_accuracy: 0.8562
Epoch 6/20
200/200 [==============================] - 35s 175ms/step - loss: 0.4523 - accuracy: 0.8662 - val_loss: 0.4541 - val_accuracy: 0.8775
Epoch 7/20
200/200 [==============================] - 35s 177ms/step - loss: 0.4039 - accuracy: 0.8797 - val_loss: 0.4424 - val_ac

In [51]:
#model evaluation on validation set
ds_cnn_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer='adam')
val_loss= ds_cnn_model.evaluate(val_ds)

25/25 [==============================] - 1s 41ms/step - loss: 1.0274


In [52]:
#model test on the test set
test_loss= ds_cnn_model.evaluate(test_ds)

25/25 [==============================] - 1s 41ms/step - loss: 1.1198
